In [180]:
import pandas as pd
import numpy as np
import re
from math import ceil, floor
from random import randint
import random
from datetime import datetime

In [181]:
## Use when data is not clean

un_filtered_csv = pd.read_excel('activ.xlsx')
cols = un_filtered_csv.columns
filtered = un_filtered_csv[[cols[0], cols[1], cols[-1]]]
filtered = pd.DataFrame(filtered.values, columns=["IMEI", "Name", "Time"])

In [182]:
csv_data = pd.read_csv('data.csv', sep=',') if filtered is None else filtered

In [183]:
csv_data
unique_names = csv_data.Name.unique()
unique_names

array(['Redmi Note 9 Pebble Grey 4GB RAM 128GB ROM',
       'Redmi 9A Nature Green 3GB RAM 32GB ROM',
       'Redmi 9 Carbon Black 4GB RAM 64GB ROM',
       'Redmi 9 Sky Blue 4GB RAM 64GB ROM',
       'Redmi 9i Midnight Black 4GB RAM 64GB ROM',
       'Redmi 9i Sea Blue 4GB RAM 64GB ROM',
       'Redmi 9i Nature Green 4GB RAM 64GB ROM',
       'Redmi 9 Power Mighty Black 4GB RAM 64GB ROM',
       'Redmi 9 Power Electric Green 4GB RAM 64GB ROM',
       'Redmi 9 Power Blazing Blue 4GB RAM 64GB ROM'], dtype=object)

In [184]:
np.random.shuffle(csv_data.values)

In [185]:
keywords = ["Redmi", "9", "9A", "9i", "Power", "2GB", "3GB", "4GB", "128GB", "64GB", "32GB" ]
def model_name_reducer(name):
    spl = name.split(" ")
    gen =""
    for s in spl:
        if s in keywords:
            gen += s +" "
    return gen

In [186]:
def freeze_price_list(price_list):
    data = []
    today = datetime.now().date()
    for key in price_list.keys():
        data.append([f"{today.day}/{today.month}/{today.year}", key, price_list[key]])
    ls = pd.DataFrame(data,columns=['Date', 'Name', 'Price'])
    ls.to_csv('price_list.csv')
    return ls

In [187]:
price_list = {}
def ask_price():
    today = datetime.now().date()
    try:
        real = pd.read_csv('price_list.csv', sep=',')
        ls = real.query(f"Date=='{today.day}/{today.month}/{today.year}'")
    except FileNotFoundError:
        ls = None
    for item in unique_names:
        item_query = None
        if isinstance(ls, pd.DataFrame) and not ls.empty:
            item_query = ls.query(f'Name=="{item}"')
#         print(item_query)
        if not (isinstance(item_query, pd.DataFrame) and not item_query.empty):
            price = float(input(f"{item} price: "))
        else:
            price = float(ls.values[0][-1])
        price_list[item] = price
    l = freeze_price_list(price_list)
    return l
price_list = ask_price()

Redmi Note 9 Pebble Grey 4GB RAM 128GB ROM price: 13000
Redmi 9A Nature Green 3GB RAM 32GB ROM price: 7500
Redmi 9 Carbon Black 4GB RAM 64GB ROM price: 8800
Redmi 9 Sky Blue 4GB RAM 64GB ROM price: 8800
Redmi 9i Midnight Black 4GB RAM 64GB ROM price: 8000
Redmi 9i Sea Blue 4GB RAM 64GB ROM price: 8000
Redmi 9i Nature Green 4GB RAM 64GB ROM price: 8000
Redmi 9 Power Mighty Black 4GB RAM 64GB ROM price: 10500
Redmi 9 Power Electric Green 4GB RAM 64GB ROM price: 10500
Redmi 9 Power Blazing Blue 4GB RAM 64GB ROM price: 10500


In [188]:
names = pd.read_csv('names.csv',sep=',')
names

,Unnamed: 0,first_name,last_name,rel
0,0,pradyum,shukla,H
1,1,nitin,chaurasia,H
2,2,sailesh,rawat,H
3,3,dinesh,singh,H
4,4,kamlesh,gupta,H
...,...,...,...,...
97,97,divyansh,verma,H
98,98,bhuvan,patel,H
99,99,krishna,mishra,H
100,100,pawan,gupta,H


In [189]:
csv_data.values

array([[869770045188636, 'Redmi Note 9 Pebble Grey 4GB RAM 128GB ROM',
        Timestamp('2021-02-25 15:19:28')],
       [865650050500426, 'Redmi 9A Nature Green 3GB RAM 32GB ROM',
        Timestamp('2021-02-25 19:02:44')],
       [867830055519682, 'Redmi 9A Nature Green 3GB RAM 32GB ROM',
        Timestamp('2021-02-25 13:39:38')],
       [867830055460069, 'Redmi 9A Nature Green 3GB RAM 32GB ROM',
        Timestamp('2021-02-25 16:38:36')],
       [866191058198979, 'Redmi 9 Carbon Black 4GB RAM 64GB ROM',
        Timestamp('2021-02-25 17:05:31')],
       [866191058205634, 'Redmi 9 Carbon Black 4GB RAM 64GB ROM',
        Timestamp('2021-02-25 17:45:01')],
       [866191058228453, 'Redmi 9 Carbon Black 4GB RAM 64GB ROM',
        Timestamp('2021-02-25 17:42:49')],
       [866191058232612, 'Redmi 9 Carbon Black 4GB RAM 64GB ROM',
        Timestamp('2021-02-25 17:12:00')],
       [866191058236290, 'Redmi 9 Carbon Black 4GB RAM 64GB ROM',
        Timestamp('2021-02-25 17:37:38')],
       [866

In [190]:
price_dict= {}
def assign_dict():
    for price in price_list.values:
        price_dict[price[1]] = price[-1]
assign_dict()

def get_price(model):
    return round(price_dict[model]/(1+(18/100)),2)

def reduced_name(array):
    return f"{array[1].title()} {array[2].title()}" 

def get_roundoff(num):
    ceiled = int(num)
    oneth = int(str(ceiled)[-1])
    if oneth == 5 or oneth == 0:
        return ceiled
    factor = 10 - oneth if oneth != 0 and oneth != 5 and oneth > 5 else (-1 * oneth)
    return ceiled + factor

def serial_no(start, stop, last=None):
    num = last if last != None else 12657
    return num + randint(start, stop)

In [191]:
from datetime import datetime
# noned = []
class Voucher:
    def __init__(self, name, array, index):
        self.index = index
        self.name = name
        self.serial = 0
        self.imei = array[0]
        self.model = array[1]
        self.model_name = model_name_reducer(self.model)
#         print(self.model)
        self.price = float(get_price(self.model))
        self.cgst =  round(self.price * 9/100,2)
        self.sgst =  round(self.price * 9/100, 2)
        self.total_price = round(self.price + self.cgst + self.sgst,2)
        self.roundoff = round(float(get_roundoff(self.total_price)) - self.total_price,2)
        self.grand_total = round(self.total_price + self.roundoff,2)
        self.time = datetime.strptime(array[2], "%m/%d/%Y %H:%M") if isinstance(array[2],str) else array[2].to_pydatetime()
    def get_time(self):
        return self.time.strftime("%d-%m-%Y")
    def getArray(self, serial):
        self.serial = serial
        return [self.serial,self.name, self.imei, model_name_reducer(self.model) , self.get_time(), self.price, self.cgst, self.sgst, self.roundoff, self.total_price, self.grand_total]

In [192]:
voucher_list =[]
names_values = names.values
for index,data in enumerate(csv_data.values):
#     print(self.model)
    voucher_list.append(Voucher(reduced_name(names_values[index]), data, index))
    


In [193]:
voucher_data = []
for index, data in enumerate(voucher_list):
    voucher_data.append(data.getArray(serial_no(1,10, last=voucher_list[index -1].serial if index > 0 else None)))

In [194]:
frame = pd.DataFrame(voucher_data, columns=['Serial','Name', 'IMEI', 'Item', 'Time', 'Price','CGST', 'SGST','Round Off', 'Total', 'Grand Total'])

In [195]:
frame
# frame.to_csv('bills.csv')

,Serial,Name,IMEI,Item,Time,Price,CGST,SGST,Round Off,Total,Grand Total
0,12666,Pradyum Shukla,869770045188636,Redmi 9 4GB 128GB,25-02-2021,11016.95,991.53,991.53,-0.01,13000.01,13000.0
1,12672,Nitin Chaurasia,865650050500426,Redmi 9A 3GB 32GB,25-02-2021,6355.93,572.03,572.03,0.01,7499.99,7500.0
2,12676,Sailesh Rawat,867830055519682,Redmi 9A 3GB 32GB,25-02-2021,6355.93,572.03,572.03,0.01,7499.99,7500.0
3,12681,Dinesh Singh,867830055460069,Redmi 9A 3GB 32GB,25-02-2021,6355.93,572.03,572.03,0.01,7499.99,7500.0
4,12689,Kamlesh Gupta,866191058198979,Redmi 9 4GB 64GB,25-02-2021,7457.63,671.19,671.19,-0.01,8800.01,8800.0
...,...,...,...,...,...,...,...,...,...,...,...
96,13158,Punit Gupta,861086056404566,Redmi 9 Power 4GB 64GB,25-02-2021,8898.31,800.85,800.85,-0.01,10500.01,10500.0
97,13161,Divyansh Verma,861086056459792,Redmi 9 Power 4GB 64GB,25-02-2021,8898.31,800.85,800.85,-0.01,10500.01,10500.0
98,13168,Bhuvan Patel,861086056460857,Redmi 9 Power 4GB 64GB,25-02-2021,8898.31,800.85,800.85,-0.01,10500.01,10500.0
99,13172,Krishna Mishra,861086056383307,Redmi 9 Power 4GB 64GB,25-02-2021,8898.31,800.85,800.85,-0.01,10500.01,10500.0


In [196]:
from fpdf import FPDF

In [197]:
from num2words import num2words
def int_to_en(num):
    return num2words(num).title()


In [198]:
COMPANY_NAME = "JAISWAL TRADING COMPANY"
ADDRESS = "SHOP NO.15, ATM LANE, BALDEV PLAZA, GOLGHAR"
CITY="Gorakhpur"
GSTIN="09AFCPJ3314B1Z3"

class VoucherPDF:
    def __init__(self, voucher, pdf):
        self.voucher = voucher
        self.pdf = pdf
    def render(self):
        self.pdf.add_page()
        self.header()
        self.body()
        self.footer()
#         self.pdf.output('out.pdf')
    
    def header(self):
        ## Juridiction
        self.pdf.set_font('Arial', 'U', 8)
        self.pdf.cell(0,10, 'SUBJECTED TO GORAKHPUT JURISDICTION', 0, 1, 'C')
        
        ## Invoice No
        self.pdf.set_font('Arial', '', 8)
        self.pdf.cell(13,10,'Invoice No. ', 0,0 ,'C')
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(8,10, str(self.voucher.serial), 0,0,'C')
#         self.pdf.cell(0,6,'',0,1,'C')
        ## Date
        self.pdf.set_font('Arial', '', 8)
        self.pdf.cell(86,10, '', 0,0, 'R')
        self.pdf.cell(12,10, 'Date ',0,0,'C')
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(12,10, self.voucher.get_time(),0,0,'C')
        
        #Name of My Company
        self.pdf.cell(0,8,'',0,1,'C')
        self.pdf.set_font('Arial', 'B', 10)
        self.pdf.cell(0,4, COMPANY_NAME,0,1,'C')
        self.pdf.set_font('Arial', '', 8)
        self.pdf.cell(0,4, ADDRESS,0,1,'C')
        self.pdf.cell(0,4, CITY,0,1,'C')
        self.pdf.cell(0,4, f'GSTIN: {GSTIN}',0,1,'C')
        self.pdf.set_font('Arial', 'U', 8)
        self.pdf.cell(0,4, 'State Name: Uttar Pradesh, Code: 09',0,1,'C')
        
        # Tax Invoice
        self.pdf.cell(0,4,'',0,1,'C')
        self.pdf.set_font('Arial', 'B', 10)
        self.pdf.cell(0,4, 'Tax Invoice',0,1,'C')
        
        # Name
        self.pdf.cell(0,6,'',0,1,'C')
        self.pdf.set_font('Arial', 'B', 10)
        self.pdf.cell(0,4, self.voucher.name.encode('latin-1', 'replace').decode('latin-1') if not "?" in self.voucher.name else random.choice(["Krishna", "Adil", "Rama", "Rsihab"]) ,0,1,'C')
        self.pdf.set_font('Arial', '', 6)
        self.pdf.cell(0,4, 'Gorakhpur',0,1,'C')
        
    
    def body(self):
        self.pdf.cell(0,4,'', 0,1,'C')
        
        ### body header
        self.pdf.set_font('Arial', '', 6)
        self.pdf.cell(55,8,'Description of Goods', 1,0,'C')
        self.pdf.cell(20,8,'Quantity', 1,0,'C')
        self.pdf.cell(20,8,'Rate', 1,0,'C')
        self.pdf.cell(10,8,'per', 1,0,'C')
        self.pdf.cell(30,8,'Amount', 1,0,'C')
        
        self.pdf.cell(0,8,'',0,1,'C')
        self.pdf.line(10,78,10,120)
        
        # Item detail
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(55,8, self.voucher.model_name,0,0,'L')
        self.pdf.line(65,78,65,120)
        self.pdf.cell(20,8,'1 pcs', 0,0,'C')
        self.pdf.line(85,78,85,120)
        self.pdf.cell(20,8,f'{self.voucher.price}', 0,0,'C')
        self.pdf.line(105,78,105,120)
        self.pdf.cell(10,8,'pcs', 0,0,'C')
        self.pdf.line(115,78,115,120)
        self.pdf.cell(30,8,f'{self.voucher.price}', 0,0,'R')
        self.pdf.line(145,78,145,120)
        self.pdf.cell(0,4,'',0,1,'C')
        self.pdf.set_font('Arial', 'BI', 8)
        self.pdf.cell(35,8, str(self.voucher.imei),0,1,'C')

        ## Taxes
        #CGST
        self.pdf.cell(0,8,'',0,1,'C')
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(55,8, 'CGST',0,0,'R')
        self.pdf.cell(20,8,'', 0,0,'C')
        self.pdf.cell(20,8,'', 0,0,'C')
        self.pdf.cell(10,8,'', 0,0,'C')
        self.pdf.cell(30,8,f'{self.voucher.cgst}', 0,0,'R')
        
        #SGST
        self.pdf.cell(0,4,'',0,1,'C')
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(55,8, 'SGST',0,0,'R')
        self.pdf.cell(20,8,'', 0,0,'C')
        self.pdf.cell(20,8,'', 0,0,'C')
        self.pdf.cell(10,8,'', 0,0,'C')
        self.pdf.cell(30,8,f'{self.voucher.sgst}', 0,0,'R')
        
        # RoundOFF
        self.pdf.cell(0,4,'',0,1,'C')
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(55,8, 'Round off',0,0,'R')
        self.pdf.cell(20,8,'', 0,0,'C')
        self.pdf.cell(20,8,'', 0,0,'C')
        self.pdf.cell(10,8,'', 0,0,'C')
        self.pdf.cell(30,8,f'{self.voucher.roundoff}', 0,0,'R')
        
        ## Total
        self.pdf.cell(0,10,'',0,1,'C')
        self.pdf.set_font('Arial', '', 6)
        self.pdf.cell(55,6,'Total', 1,0,'R')
        self.pdf.cell(20,6,'1 pcs', 1,0,'C')
        self.pdf.cell(20,6,'', 1,0,'C')
        self.pdf.cell(10,6,'', 1,0,'C')
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(30,6, f'{self.voucher.grand_total}', 1,0,'R')
        
#         self.pdf.cell(0,8,'',0,1,'C')
        self.pdf.line(10,78,10,120)
    
    def footer(self):
        self.pdf.cell(0,6,'',0,1,'C')
        self.pdf.set_font('Arial', '', 6)
        self.pdf.cell(55,6, 'Amount Chargeable(words)',0,0,'L')
        self.pdf.cell(20,6,'', 0,0,'C')
        self.pdf.cell(20,6,'', 0,0,'C')
        self.pdf.cell(10,6,'', 0,0,'C')
        self.pdf.cell(30,6,f'E & O.E', 0,1,'R')
        
        # In words
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(0,4, f'{int_to_en(self.voucher.grand_total)}',0,1,'L')
        
        ## Declaration
        self.pdf.cell(0,10,'',0,1,'C')
        self.pdf.set_font('Arial', 'U', 8)
        self.pdf.cell(10,6,'Declaration', 0,0, 'L')
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(0,6,f'for {COMPANY_NAME}', 0,1, 'R')
        
        self.pdf.set_font('Arial', '', 8)
        self.pdf.cell(0,4, 'We declare that this invoice shows the', 0,1, 'L')
        self.pdf.set_font('Arial', '', 6)
        self.pdf.cell(0,4, 'actual price of goods described and that all particulars are true and correct.', 0,1, 'L')
        
        self.pdf.set_font('Arial', 'B', 6)
        self.pdf.cell(0,6,'Authorised Signatory', 0,1, 'R')
        
        self.pdf.cell(0,10,'',0,1,'C')
        self.pdf.set_font('Arial', 'U', 8)
        self.pdf.cell(0,10,'This is a Computer Generated Invoice',0,1,'C')
        
        
        
        
        

In [200]:

pdf = FPDF(orientation='P', format='A5')
for voucher in voucher_list:
    VoucherPDF(voucher, pdf).render()
pdf.output('REDMI_BILL_MERGE.pdf')

''